In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

In [3]:
T = 0.5
K = 100.
S0 = 100.
sigma = 0.2
p = 1
M = np.eye(p)
r = 0.04
mu = 0.06
R = 0.06
Q = 0.

In [4]:
N = 10000
m = 8
RF_n_trees = 200
RF_max_leaf_nodes = 700
M_run = 20
n_picard = 5

In [5]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test = BsdeHD(T, K, M, mu, Q, sigma, S0, r, R)
    a[i] = test.get_price(N, m, option_payoff='geometric',
                          RF_max_leaf_nodes=300,
                          RF_min_samples_leaf=0.05,
                          oType='European',n_picard=n_picard, 
                         RF_warm_start = False, 
                         RF_max_depth=10)
    elapsed = time.time() - start
    print (a[i])
    timing[i] = round(elapsed, 3)

7.10906240216
7.1340763006
7.03954904063
7.07803044709
7.1550886173
7.1989158526
7.18610206459
7.14277051244
7.29258724913
7.05571904482
7.10135605019
7.11738536616
7.23657546887
7.28534993456
7.02883218225
7.09501821966
7.13295283903
7.08680361259
7.28495788357
7.07327948945


In [7]:
timing.mean()

7.1417206288845225

In [8]:
F_max_depth=None,
                  RF_min_samples_split=2,
                  RF_min_samples_leaf=1,

IndentationError: unexpected indent (<ipython-input-8-593aa93af6e2>, line 2)

In [9]:
import pandas as pd

In [12]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [Q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [13]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard','Number of trees', 'Max Leaf nodes']
svalues = [N, m, M_run, n_picard, RF_n_trees, RF_max_leaf_nodes]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,10000
1,m_discretization,8
2,N_run,20
3,n_picard,5
4,Number of trees,200
5,Max Leaf nodes,700


In [14]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [15]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [16]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,7.1417
1,std,0.0815
2,95% confidence interval,"[7.1337, 7.1497]"
3,min,7.0288
4,max,7.2926


In [17]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_rf.csv')

## Fitting the parameters

In [ ]:
N = 10000
m = 6
RF_n_trees = 200
RF_max_leaf_nodes = 1000
M_run = 20
n_picard = 3

In [ ]:
range_leaf = len(list(range(50, 10000, 50)))
b = np.zeros([range_leaf, 10])
timing = np.zeros_like(b)
for i, leaf in enumerate(list(range(50, 10000, 50))): 
    for j in range(10): 
        start=time.time()
        test = BSDE(S0, K, T, mu, sigma, q)
        b[i, j] = round(test.get_price_RF(R, r, N, m, n_picard=n_picard, RF_max_leaf_nodes=leaf, RF_n_tree=200), 4)
        timing[i, j] = round(time.time()- start, 3)
    print (b[i])   

In [ ]:
N = 100000

In [ ]:
range_leaf = len(list(range(1000, 50000, 1000)))
c = np.zeros([range_leaf, 10])
timing = np.zeros_like(c)
for i, leaf in enumerate(list(range(1000, 50000, 1000))): 
    for j in range(10): 
        start=time.time()
        test = BSDE(S0, K, T, mu, sigma, q)
        c[i, j] = round(test.get_price_RF(R, r, N, m, n_picard=n_picard, RF_max_leaf_nodes=leaf, RF_n_tree=200), 4)
        timing[i, j] = round(time.time()- start, 3)
    print (c[i])  

In [ ]:
means_b = b.mean(axis=1)

In [ ]:
means_c = c.mean(axis=1)

In [ ]:
means_c

c

In [ ]:
columns = ["run {}".format(i + 1) for i in range(10)]

In [ ]:
index = ["max_leaf_nodes = {}".format(j) for j in list(range(50, 10000, 50))] 
index_c = ["max_leaf_nodes = {}".format(j) for j in list(range(1000, 50000, 1000))]

In [ ]:
matrix_b = pd.DataFrame(data=b, columns=columns, index=index)

In [ ]:
matrix_c = pd.DataFrame(data=c, columns=columns, index=index_c)

In [ ]:
matrix_b.to_csv("price_against_leafs_1000.csv")

In [ ]:
matrix_c.to_csv("price_against_leafs_100000.csv")